<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/work_in_progress/scripts/instruct_model/100_create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction prompts generation

**Author:** Eva Rombouts  
**Date:**   

### Description


In [1]:
# The GenCareAIUtils package contains the class GenCareAISetup. GenCareAISetup detects the environment (Google Colab or local), handles file paths, and retrieves API keys. The package simplifies environment setup.
!pip install GenCareAI
from GenCareAI.GenCareAIUtils import GenCareAISetup

setup = GenCareAISetup()

if setup.environment == 'Colab':
    !pip install -q datasets sentence-transformers langchain langchain_openai langchain_community

verbose = True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.


In [2]:
# Import libraries
import os
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

import torch
from sentence_transformers import SentenceTransformer

from tqdm.autonotebook import tqdm, trange
from datasets import load_dataset, Dataset, DatasetDict

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Set parameters
seed = 6
path_hf_records = "ekrombouts/Gardenia_records"
path_hf_clients = "ekrombouts/Gardenia_clients"
path_hf_instruct = "ekrombouts/Gardenia_instruct_dataset"
commit_message = "Instruct dataset"

# File paths for saving/loading the embeddings
fn_male_context_embeddings = setup.get_file_path('data/care_pal/male_context_embeddings.pt')
fn_female_context_embeddings = setup.get_file_path('data/care_pal/female_context_embeddings.pt')
fn_male_instruction_embeddings = setup.get_file_path('data/care_pal/male_instruction_embeddings.pt')
fn_female_instruction_embeddings = setup.get_file_path('data/care_pal/female_instruction_embeddings.pt')
fn_responses = setup.get_file_path('data/care_pal/context_instruction_pairs_with_responses.pkl')

# Set additional parameters
num_general_prompts = 250
k_instructions=2
k_contexts=50
sep_line = 50*'-'

# Set seed for reproducibility
random.seed(seed)

## Load and preprocess data

In [4]:
# Load datasets from Hugging Face and preprocess
def load_and_preprocess_data():
    dataset_records = load_dataset(path_hf_records)
    dataset_clients = load_dataset(path_hf_clients)

    df_records = dataset_records['train'].to_pandas()
    df_clients = dataset_clients['train'].to_pandas()

    def determine_client_gender(row):
        name = row["name"]
        if "Mevrouw" in name:
            return "female"
        elif "Meneer" in name:
            return "male"
        else:
            return "unknown"

    df = (df_records
          .dropna()
          .assign(week=lambda df: pd.to_datetime(df['datetime']).dt.to_period('W').dt.to_timestamp())  # Add 'week' column
          .groupby(['client_id', 'week'])
          .agg({'note': lambda x: '\n'.join(x)})  # Concatenate 'note' values
          .reset_index()
          .rename(columns={'note': 'weeknotes'})
          .merge(df_clients[['client_id', 'name']], on='client_id', how='left')  # Merge with client name
          .assign(gender=lambda df: df.apply(determine_client_gender, axis=1))  # Determine gender
         )
    return df, df_records

df, df_records = load_and_preprocess_data()

if verbose:
  print(f"Rows in original df: {df_records.shape[0]}, rows in processed df: {df.shape[0]}\n")
  print(f"SAMPLES{sep_line}\n{df.sample(3)}\n")
  print(f"\nContext column (weeknotes) example:{sep_line}\n{df['weeknotes'].iloc[0]}")
  print(f"\nPercentage gender:{sep_line}\n{df['gender'].value_counts(normalize=True)}")


Rows in original df: 28478, rows in processed df: 1681

SAMPLES--------------------------------------------------
    client_id       week                                          weeknotes  \
326    cro015 2022-12-12  Dhr. had vanmorgen moeite met het gebruik van ...   
667    iri001 2023-02-06  Client reageerde niet goed op medicatie vanoch...   
836    iri009 2022-10-17  Client had vanochtend moeite met opstaan en aa...   

                        name gender  
326        Meneer Gerrit Vos   male  
667  Meneer Arnold den Bosch   male  
836   Meneer Ferdinand Steen   male  


Context column (weeknotes) example:--------------------------------------------------
De cliënt is vandaag gearriveerd in het verpleeghuis. Hij is voorgesteld aan het personeel en de medebewoners. Het zorgplan is opgesteld en de benodigde hulp bij persoonlijke verzorging is gepland.
Cliënt heeft hulp nodig gehad bij aankleden en eten. Hij kon korte afstanden lopen met behulp van een rollator.
Cliënt leek wat ver

## Create instruction prompts

In [5]:
# Define instruction prompts for male, female, and general contexts
instructions_male = [
    "Beschrijf lichamelijke klachten",
    "Hoe voelt de patiënt zich?",
    "Welke ziektes heeft cliënt?",
    "Beschrijf de klachten van meneer",
    "Heeft deze cliënt pijn?",
    "Welke ongemakken ervaart dhr?",
    "Welke behandeling is ingezet?",
    "Beschrijf of er wonden of huidproblemen zijn",
    "Beschrijf de benodigde ADL hulp",
    "Beschrijf bijzonderheden over eten en drinken",
    "Welke hulp heeft dhr nodig bij wassen en aankleden?",
    "Geef aan welke hulp wordt geboden bij eten en drinken",
    "Wordt meneer geholpen bij douchen?",
    "Hoe wordt de ADL gedaan?",
    "Beschrijf de mobiliteit van meneer",
    "Welk loophulpmiddel gebruikt cliënt?",
    "Beschrijf de mate van valgevaar",
    "Welke hulp wordt geboden bij de mobiliteit?",
    "Beschrijf de daginvulling van meneer",
    "Doet ct mee aan activiteiten?",
    "Hoe verlopen de nachten?",
    "Heeft meneer lekker geslapen?",
    "Geef aan of er stemmingsklachten zijn",
    "Beschrijf gedragsproblemen",
    "Hoe is de cognitie van meneer?",
]

instructions_female = [
    "Beschrijf lichamelijke klachten",
    "Hoe voelt de patiënt zich?",
    "Welke ziektes heeft cliënte?",
    "Beschrijf de klachten van mevrouw",
    "Heeft deze cliënte pijn?",
    "Welke ongemakken ervaart mw?",
    "Welke behandeling is ingezet?",
    "Beschrijf of er wonden of huidproblemen zijn",
    "Beschrijf de benodigde ADL hulp",
    "Beschrijf bijzonderheden over eten en drinken",
    "Welke hulp heeft mw nodig bij wassen en aankleden?",
    "Geef aan welke hulp wordt geboden bij eten en drinken",
    "Wordt mevrouw geholpen bij douchen?",
    "Hoe wordt de ADL gedaan?",
    "Beschrijf de mobiliteit van mevrouw",
    "Welk loophulpmiddel gebruikt cliënte?",
    "Beschrijf de mate van valgevaar",
    "Welke hulp wordt geboden bij de mobiliteit?",
    "Beschrijf de daginvulling van mevrouw",
    "Doet cte mee aan activiteiten?",
    "Hoe verlopen de nachten?",
    "Heeft mevrouw lekker geslapen?",
    "Geef aan of er stemmingsklachten zijn",
    "Beschrijf gedragsproblemen",
    "Hoe is de cognitie van mevrouw?",
]

instructions_general = [
    "Geef twee belangrijke punten waarop moet worden geobserveerd en gerapporteerd",
    "Noem de aandachtspunten voor het zorgpersoneel",
    "Welke acties moet het zorgteam nemen op basis van deze rapportages?",
    "Vat de rapportages kort en bondig samen",
]

# Combine all instruction prompts
instructions = instructions_male + instructions_female + instructions_general

if verbose:
    print(f"Number of male instructions: {len(instructions_male)}")
    print(f"Number of female instructions: {len(instructions_female)}")
    print(f"Number of general instructions: {len(instructions_general)}")
    print(f"Total number of instructions: {len(instructions)}\n")

Number of male instructions: 25
Number of female instructions: 25
Number of general instructions: 4
Total number of instructions: 54



## Get contexts

In [6]:
# Create lists of contexts
male_contexts = df[df['gender'] == 'male']['weeknotes'].tolist()
female_contexts = df[df['gender'] == 'female']['weeknotes'].tolist()
contexts = male_contexts + female_contexts

if verbose:
    print(f"Number of male contexts: {len(male_contexts)}")
    print(f"Number of female contexts: {len(female_contexts)}")
    print(f"Total number of contexts: {len(contexts)}")


Number of male contexts: 842
Number of female contexts: 839
Total number of contexts: 1681


## Load or Generate Embeddings

In [7]:
# Load the embeddings model
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Function to load or generate embeddings
def load_or_generate_embeddings(file_path, data, model):
    if os.path.exists(file_path):
        print(f"Loading embeddings from {file_path}")
        embeddings = torch.load(file_path, weights_only=True)
    else:
        print(f"Generating embeddings for {file_path}")
        embeddings = model.encode(
            sentences=data,
            convert_to_tensor=True,
            show_progress_bar=True
        )
        torch.save(embeddings, file_path)
    return embeddings

# Load or generate embeddings for male and female contexts and instructions
male_context_embeddings = load_or_generate_embeddings(fn_male_context_embeddings, male_contexts, model)
female_context_embeddings = load_or_generate_embeddings(fn_female_context_embeddings, female_contexts, model)
male_instruction_embeddings = load_or_generate_embeddings(fn_male_instruction_embeddings, instructions_male, model)
female_instruction_embeddings = load_or_generate_embeddings(fn_female_instruction_embeddings, instructions_female, model)

if verbose:
    print(f"\nLength of male_context_embeddings: {len(male_context_embeddings)}")
    print(f"\nShape of the first embedding: {male_context_embeddings[0].shape}")
    print(f"\nFirst embedding:\n{male_context_embeddings[0][:20]}...")  # Only shows the first values


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading embeddings from /content/drive/My Drive/Colab Notebooks/GenCareAI/data/care_pal/male_context_embeddings.pt
Loading embeddings from /content/drive/My Drive/Colab Notebooks/GenCareAI/data/care_pal/female_context_embeddings.pt
Loading embeddings from /content/drive/My Drive/Colab Notebooks/GenCareAI/data/care_pal/male_instruction_embeddings.pt
Loading embeddings from /content/drive/My Drive/Colab Notebooks/GenCareAI/data/care_pal/female_instruction_embeddings.pt

Length of male_context_embeddings: 842

Shape of the first embedding: torch.Size([512])

First embedding:
tensor([-0.0341, -0.0044, -0.0154,  0.0173,  0.0465,  0.0070, -0.0402, -0.0216,
         0.0473,  0.0299, -0.0344,  0.0743,  0.0373, -0.0213, -0.0308,  0.0163,
        -0.0564, -0.0126, -0.0306,  0.0160])...


##Calculate Cosine Similarity and Process Pairs

In [8]:
# Function to calculate top-k similarities
def get_top_k_indices(cosine_sim_matrix, k, dim):
    return torch.topk(torch.tensor(cosine_sim_matrix), k=k, dim=dim)

def process_context_instruction_pairs(cosine_sim_matrix, contexts, instructions, k_instructions=2, k_contexts=50):
    context_instruction_pairs = []

    # Top K instructions per context
    top_k_instructions_for_contexts = get_top_k_indices(cosine_sim_matrix, k=k_instructions, dim=1)

    # Top K contexts per instruction
    top_k_contexts_for_instructions = get_top_k_indices(cosine_sim_matrix, k=k_contexts, dim=0)

    # Least fitting context per instruction
    worst_contexts_for_instructions = torch.argmin(torch.tensor(cosine_sim_matrix), dim=0)

    # Add top K instructions for each context
    for i, top_instruction_indices in enumerate(top_k_instructions_for_contexts.indices):
        for idx in top_instruction_indices:
            context_instruction_pairs.append({
                "context": contexts[i],
                "instruction": instructions[idx.item()],
                "similarity": cosine_sim_matrix[i, idx.item()],
                "relationship_type": "top instructions for context"
            })

    # Add top K contexts for each instruction
    for j, top_context_indices in enumerate(top_k_contexts_for_instructions.indices.T):
        for idx in top_context_indices:
            context_instruction_pairs.append({
                "context": contexts[idx.item()],
                "instruction": instructions[j],
                "similarity": cosine_sim_matrix[idx.item(), j],
                "relationship_type": "top contexts for instruction"
            })

    # Add least fitting context for each instruction
    for j, worst_context_idx in enumerate(worst_contexts_for_instructions):
        context_instruction_pairs.append({
            "context": contexts[worst_context_idx.item()],
            "instruction": instructions[j],
            "similarity": cosine_sim_matrix[worst_context_idx.item(), j],
            "relationship_type": "worst context for instruction"
        })

    return context_instruction_pairs

## Combine male & female pairs

In [9]:
# Process male and female datasets
male_cosine_sim_matrix = cosine_similarity(male_context_embeddings, male_instruction_embeddings)
female_cosine_sim_matrix = cosine_similarity(female_context_embeddings, female_instruction_embeddings)

context_instruction_pairs_male = process_context_instruction_pairs(
    cosine_sim_matrix=male_cosine_sim_matrix,
    contexts=male_contexts, instructions=instructions_male,
    k_instructions=k_instructions,
    k_contexts=k_contexts
)

context_instruction_pairs_female = process_context_instruction_pairs(
    cosine_sim_matrix=female_cosine_sim_matrix,
    contexts=female_contexts,
    instructions=instructions_female,
    k_instructions=k_instructions,
    k_contexts=k_contexts)

# Combine male and female pairs
context_instruction_pairs = context_instruction_pairs_male + context_instruction_pairs_female

# Add general instructions to context-instruction pairs
random.seed(seed)
for instruction in instructions_general:
    sampled_contexts = random.sample(contexts, num_general_prompts)
    for context in sampled_contexts:
        context_instruction_pairs.append({
            "context": context,
            "instruction": instruction,
            "similarity": 0.0,
            "relationship_type": "general"
        })

if verbose:
    print(f"Cosine similarity matrix shape - male: {male_cosine_sim_matrix.shape}")
    print(f"Cosine similarity matrix shape - female: {female_cosine_sim_matrix.shape}")

Cosine similarity matrix shape - male: (842, 25)
Cosine similarity matrix shape - female: (839, 25)


In [10]:
# Convert context_instruction_pairs into a DataFrame
df_context_instruction_pairs = pd.DataFrame(context_instruction_pairs)

if verbose:
    print(f"SAMPLES\n{df_context_instruction_pairs.sample(3)}\n")
    print("INFO")
    print(df_context_instruction_pairs.info())
    print(f"\nVALUE COUNTS\n{df_context_instruction_pairs['instruction'].value_counts()}\n")


SAMPLES
                                                context  \
1380  Cliënt maakte vanmorgen zelf grapjes en leek g...   
40    Client vertoonde vanochtend agressief gedrag e...   
2446  Cliënt kreeg vanmorgen frequente begeleiding b...   

                               instruction  similarity  \
1380               Heeft deze cliënt pijn?    0.299746   
40                 Heeft deze cliënt pijn?    0.280207   
2446  Welk loophulpmiddel gebruikt cliënt?    0.195439   

                 relationship_type  
1380  top instructions for context  
40    top instructions for context  
2446  top contexts for instruction  

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6912 entries, 0 to 6911
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   context            6912 non-null   object 
 1   instruction        6912 non-null   object 
 2   similarity         6912 non-null   float64
 3   relationship_type

In [11]:
# Remove duplicates of columns 'context', 'instruction' en 'similarity'
df_context_instruction_pairs = df_context_instruction_pairs.drop_duplicates(subset=['context', 'instruction', 'similarity'])

if verbose:
    print(f"Num rows after dropping duplicates: {df_context_instruction_pairs.shape[0]}\n")
    print("Info")
    print(df_context_instruction_pairs.info())


Num rows after dropping duplicates: 5725

Info
<class 'pandas.core.frame.DataFrame'>
Index: 5725 entries, 0 to 6911
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   context            5725 non-null   object 
 1   instruction        5725 non-null   object 
 2   similarity         5725 non-null   float64
 3   relationship_type  5725 non-null   object 
dtypes: float64(1), object(3)
memory usage: 223.6+ KB
None


In [13]:
def create_prompt(row):
    return f"""Lees onderstaande rapportages, die een periode van een client in het verpleeghuis beschrijven, en beantwoord onderstaande instructie.
Baseer je uitsluitend op de informatie die in de rapportages staat. Als er geen relevante informatie in staat, zeg dat dan. Hou je antwoord kort en bondig.

RAPPORTAGES:
{row['context']}

INSTRUCTIE:
{row['instruction']}"""

df_context_instruction_pairs['prompt'] = df_context_instruction_pairs.apply(create_prompt, axis=1)

if verbose:
    print(f"SAMPLES\n{df_context_instruction_pairs.sample(3)}\n")
    print("INFO")
    print(df_context_instruction_pairs.info())
    print(f"\nPROMPT SAMPLE:\n{df_context_instruction_pairs['prompt'].iloc[0]}")

SAMPLES
                                               context  \
854  Cliënt had vannacht onrustige dromen en was da...   
172  Meneer had vanochtend last van toenemende pijn...   
185  Meneer had vanochtend moeite met opstaan uit b...   

                                           instruction  similarity  \
854                    Beschrijf lichamelijke klachten    0.139319   
172                            Heeft deze cliënt pijn?    0.205988   
185  Geef aan welke hulp wordt geboden bij eten en ...    0.199888   

                relationship_type  \
854  top instructions for context   
172  top instructions for context   
185  top instructions for context   

                                                prompt  
854  Lees onderstaande rapportages, die een periode...  
172  Lees onderstaande rapportages, die een periode...  
185  Lees onderstaande rapportages, die een periode...  

INFO
<class 'pandas.core.frame.DataFrame'>
Index: 5725 entries, 0 to 6911
Data columns (total 5 colu

In [14]:
template = """Lees onderstaande rapportages, die een periode van een client in het verpleeghuis beschrijven, en beantwoord onderstaande instructie.
Baseer je uitsluitend op de informatie die in de rapportages staat. Als er geen relevante informatie in staat, zeg dat dan. Hou je antwoord kort en bondig.

RAPPORTAGES:
{context}

INSTRUCTIE:
{instruction}
"""

In [15]:
prompt_template = PromptTemplate(
    input_variables=["context", "instruction"],
    template=template,
)

# Initialize the language model with specified parameters
llm = ChatOpenAI(
    api_key=setup.get_openai_key(),
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-3.5-turbo-0125",
    temperature=0.3,
    presence_penalty=0.2,
)

chain = prompt_template | llm



In [16]:
if verbose:
    sample_id = 50
    sample_context = df_context_instruction_pairs['context'].iloc[sample_id]
    sample_instruction = df_context_instruction_pairs['instruction'].iloc[sample_id]

    sample_prompt = template.format(
            context=sample_context,
            instruction=sample_instruction
    )

    result = chain.invoke({"context": sample_context, "instruction": sample_instruction})

    print(sample_prompt)
    print("RELATIONSHIP TYPE")
    print(df_context_instruction_pairs['relationship_type'].iloc[sample_id])
    print("RESPONSE")
    print(result.content)


Lees onderstaande rapportages, die een periode van een client in het verpleeghuis beschrijven, en beantwoord onderstaande instructie.
Baseer je uitsluitend op de informatie die in de rapportages staat. Als er geen relevante informatie in staat, zeg dat dan. Hou je antwoord kort en bondig.

RAPPORTAGES:
Cliënt is vanochtend rustig wakker geworden en heeft ontbeten in bed. Hij was emotioneel afstandelijk en reageerde niet veel op mijn vragen. Aanbod gedaan voor hulp bij persoonlijke verzorging, maar cliënt gaf aan het zelf te willen proberen.
Tijdens de lunch had cliënt weinig eetlust en at maar een paar happen. Hierna liet hij weten moe te zijn en wilde rusten in bed. Geen tekenen van lichamelijk ongemak of pijn.
Cliënt heeft vanmiddag wat tijd doorgebracht in de recreatiezaal, maar heeft voornamelijk naar buiten gestaard. Hij leek verdrietig en kwam niet tot interactie met andere bewoners. Geen fysiotherapie vandaag.
Cliënt was vanmorgen vroeg wakker en voelde zich onrustig. Hij had la

In [17]:
# Load the previously saved dataframe if it exists, otherwise start fresh
if os.path.exists(fn_responses):
    df_context_instruction_pairs = pd.read_pickle(fn_responses)
else:
    df_context_instruction_pairs['llm_response'] = None  # Ensure the column exists

# Function to add LLM answer to the dataframe with error handling
def get_llm_answer(row, cb):
    try:
        if pd.isna(row['llm_response']):  # Process only new prompts
            result = chain.invoke({"context": row['context'], "instruction": row['instruction']}, callbacks=[cb])
            return result.content
        else:
            return row['llm_response']  # Keep existing answers
    except Exception as e:
        print(f"Error processing row {row.name}: {e}")
        return None

# Create a callback instance to track cost
with get_openai_callback() as cb:

    # Iterate over the rows and save progress intermittently
    for idx, row in df_context_instruction_pairs.iterrows():
        df_context_instruction_pairs.at[idx, 'llm_response'] = get_llm_answer(row, cb)

        # Save progress every 10 iterations
        if idx % 100 == 0:
            df_context_instruction_pairs.to_pickle(fn_responses)
            print(f"Checkpoint saved at index {idx}, total cost so far: ${cb.total_cost:.4f}")

    # Save the final result
    df_context_instruction_pairs.to_pickle(fn_responses)
    print("Processing complete and final dataframe saved.")
    print(f"Total cost: ${cb.total_cost:.4f}")

Checkpoint saved at index 0, total cost so far: $0.0000
Checkpoint saved at index 100, total cost so far: $0.0000
Checkpoint saved at index 200, total cost so far: $0.0000
Checkpoint saved at index 300, total cost so far: $0.0000
Checkpoint saved at index 400, total cost so far: $0.0000
Checkpoint saved at index 500, total cost so far: $0.0000
Checkpoint saved at index 600, total cost so far: $0.0000
Checkpoint saved at index 700, total cost so far: $0.0000
Checkpoint saved at index 800, total cost so far: $0.0000
Checkpoint saved at index 900, total cost so far: $0.0000
Checkpoint saved at index 1000, total cost so far: $0.0000
Checkpoint saved at index 1100, total cost so far: $0.0000
Checkpoint saved at index 1200, total cost so far: $0.0000
Checkpoint saved at index 1300, total cost so far: $0.0000
Checkpoint saved at index 1400, total cost so far: $0.0000
Checkpoint saved at index 1500, total cost so far: $0.0000
Checkpoint saved at index 1600, total cost so far: $0.0000
Checkpoin

In [18]:
example_ct = 63
print(df_context_instruction_pairs['prompt'].iloc[example_ct])
print("\nRESPONSE:")
print(df_context_instruction_pairs['llm_response'].iloc[example_ct])

Lees onderstaande rapportages, die een periode van een client in het verpleeghuis beschrijven, en beantwoord onderstaande instructie.
Baseer je uitsluitend op de informatie die in de rapportages staat. Als er geen relevante informatie in staat, zeg dat dan. Hou je antwoord kort en bondig.

RAPPORTAGES:
De mentale toestand van de cliënt lijkt stabiel. Morgenactiviteiten aangepast om hem meer te betrekken bij het groepsgebeuren.
Het gewichtsverlies blijft een zorg. Psycholoog ingeschakeld om te praten over mogelijke onderliggende oorzaken van eetproblemen.
Cliënt huilt vanavond bij het naar bed gaan. Troost geboden en rustgevende muziek afgespeeld om te helpen bij het kalmeren.
De cliënt lijkt vandaag wat meer alert te zijn en toont interesse in de activiteiten. Stimulatie van zintuigen ingevoerd om cognitie te bevorderen.
Gewichtsverlies blijft een probleem, ondanks alle inspanningen. Diëtist werkt aan een persoonlijk voedingsplan om de calorie-inname te verhogen.
Cliënt lijkt vanavond 

In [19]:
df = (df_context_instruction_pairs
      .loc[:, ['context', 'instruction', 'llm_response']]
      .rename(columns={'llm_response': 'response'})
     )

dataset = Dataset.from_pandas(
    df=df,
    preserve_index=False
)

# Split the dataset into training and test/validation splits
train_testvalid_split = dataset.train_test_split(test_size=0.2, seed=seed)

# Further split the test set into validation and test sets
test_valid_split = train_testvalid_split['test'].train_test_split(test_size=0.5, seed=seed)

# Create a DatasetDict object to hold the splits
dataset_dict = DatasetDict({
    'train': train_testvalid_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

# # Push the dataset to HuggingFace Hub with the specified path and commit message
# dataset_dict.push_to_hub(path_hf_instruct,
#                          commit_message=commit_message,
#                          private=True)